In [7]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
from PIL import Image


train_path = '/kaggle/input/arabic-data-set/train/train/'
test_path = '/kaggle/input/arabic-data-set/test/test/'

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        # Store the filenames and labels
        self.image_filenames = []
        self.labels = []
        for filename in os.listdir(data_dir):
            if filename.endswith('.png'):
                label = int(filename.split('_')[-1].split('.')[0]) - 1
                self.image_filenames.append(filename)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        label = self.labels[idx]
        img_path = os.path.join(self.data_dir, img_name)
        img = Image.open(img_path).convert('L')  # Convert to grayscale

        if self.transform:
            img = self.transform(img)

        return img, label


# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_image_dataset = CustomImageDataset(train_path, transform=transform)
test_dataset = CustomImageDataset(test_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size= 64, shuffle=False)
loader = DataLoader(train_image_dataset, batch_size= 64, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
class ConvolutionNNetwork(nn.Module):
    def __init__(self, output_channels, kernel_size, dropout_rate,learning_rate,padding):
        super(ConvolutionNNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, output_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(output_channels, 2 * output_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(2 * output_channels, 2 * output_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(4 * (2 * output_channels) * 4, 1000)
        self.fc2 = nn.Linear(1000, 28)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [9]:
def train_and_evaluate(model, train_loader, valid_loader, criterion, optimizer, num_epochs=15):
    val_loss_list = []
    current_patience = 0
    patience=3
    for epoch in range(num_epochs):
        model.train()
        for i, data in enumerate(loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            valid_predictions = []
            valid_labels = []
            val_loss = 0.0
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                valid_predictions.extend(predicted.cpu().numpy())
                valid_labels.extend(labels.cpu().numpy())
                val_loss += criterion(outputs, labels).item()
                
            val_loss /= len(valid_loader)
            val_loss_list.append(val_loss)
            
            accuracy = accuracy_score(valid_labels, valid_predictions)
            print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}')
            
        if epoch > 0 and val_loss < val_loss_list[epoch - 1]:
            current_patience = 0
        else:
            current_patience += 1

        if current_patience >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break

        print(f'   Val loss = ',{val_loss})
    return accuracy



In [10]:
from sklearn.metrics import accuracy_score

from sklearn.model_selection import ParameterGrid, train_test_split

param_grid = {
    'output_channels': [32, 64],
    'kernel_size': [3, 5],
    'dropout_rate': [0.2, 0.5],
    'learning_rate':[0.001,0.0001]
}

param_combinations = list(ParameterGrid(param_grid))

best_accuracy = 0.0
best_hyperparameters = {}



for params in param_combinations:
    
    if params['kernel_size'] == 3:
        params['padding'] = 1
    elif params['kernel_size'] == 5:
        params['padding'] = 2

    model = ConvolutionNNetwork(**params).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'],weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    print(f"\nTraining model with hyperparameters: {params}")
    accuracy=train_and_evaluate(model, loader, test_loader, criterion, optimizer)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = params.copy()

print(f'\nBest Test Accuracy: {best_accuracy:.4f} with hyperparameters: {best_hyperparameters}')




Training model with hyperparameters: {'dropout_rate': 0.2, 'kernel_size': 3, 'learning_rate': 0.001, 'output_channels': 32, 'padding': 1}
Epoch 1/15, Validation Accuracy: 0.7560
   Val loss =  {0.7413518406310171}
Epoch 2/15, Validation Accuracy: 0.8557
   Val loss =  {0.45389031520429646}
Epoch 3/15, Validation Accuracy: 0.8673
   Val loss =  {0.41917518962104366}
Epoch 4/15, Validation Accuracy: 0.9009
   Val loss =  {0.3233347627914177}
Epoch 5/15, Validation Accuracy: 0.9060
   Val loss =  {0.29896158134599904}
Epoch 6/15, Validation Accuracy: 0.9101
   Val loss =  {0.2980285994005653}
Epoch 7/15, Validation Accuracy: 0.9152
   Val loss =  {0.26945700468319767}
Epoch 8/15, Validation Accuracy: 0.9116
   Val loss =  {0.290233110762992}
Epoch 9/15, Validation Accuracy: 0.9208
   Val loss =  {0.26231945868370665}
Epoch 10/15, Validation Accuracy: 0.9253
   Val loss =  {0.25556695264465407}
Epoch 11/15, Validation Accuracy: 0.9137
   Val loss =  {0.2825080874674725}
Epoch 12/15, Valid